# 基础函数配置

In [1]:
import pandas as pd
import pymssql 
import datetime
import os
import json
import numpy as np
server = '192.168.0.169'
user = 'chjreport'
password = 'Chj@12345'
database = 'ChjBidb'
# 数据处理
def decode_columns(df, column_names):
    """
    对pandas数据框中的多个列进行编解码，并返回新的数据框
    
    Args:
        chuda: pandas数据框
        column_names: list，需要进行编解码的列名列表
    
    Returns:
        pandas数据框，编解码后的结果
    """
    for column_name in column_names:
        # 跳过空值
        mask = df[column_name].notnull()
        # 对指定列进行编解码
        df.loc[mask, column_name] = df.loc[mask, column_name].apply(lambda x: x.encode('latin1').decode('gbk'))
    # 返回编解码后的结果
    return df
# 动态时间
def get_dynamic_dates():
    # 获取今天的日期
    today = datetime.date.today()
    
    # 获取本月的第一天
    first_day_of_month = today.replace(day=1)
    
    # 判断今天是否在本月的7天内
    if today.day <=3:
        # 如果是，将开始时间设置为上个月的1号
        start_date = (first_day_of_month - datetime.timedelta(days=1)).replace(day=1)
    else:
        # 如果不是，将开始时间设置为本月的1号
        start_date = first_day_of_month
    
    # 将结束时间设置为昨天
    end_date = today - datetime.timedelta(days=1)
    
    # 返回开始时间和结束时间
    return start_date, end_date
start_date,end_date=get_dynamic_dates()
# url提取
# 还有这些类型的url：pages/mine/invite/index?RecoID=1492；pages/landing/special/index?redirectId=1409&RecoID=1381；pages/mine/coupons/index?RecoID=1511；pages/home/index?RecoID=1502
'''
3月自我犒赏（21530）：pages/detail/index?prodid=15289&RecoID=1498
3月炫首饰盒（57151）：pages/activity2/love/index?RecoID=1506
3月男性送礼（98161）：pages/home/index?RecoID=1519
2月秒杀专区（17583）：pages/theme/seclist/index?type=2&RecoID=1473
2月婚恋专区（18389）：pages/landing/gift/index?tabs=4&RecoID=1470
2月兑礼专区（4049）：pages/landing/gift/index?tabs=3&RecoID=1469
用户调研，触达10241人
链接：pages/home/index?RecoID=1502
提运专区：触达147839人，点击pages/landing/special/index?redirectId=1409&RecoID=1381
1-3月的我有一个紫金权益、首饰盒裂变：
紫金权益：触达36862人，点击
pages/mine/coupons/index?RecoID=1511
首饰盒裂变：触达5819人，点击
pages/mine/invite/index?RecoID=1492

这些类型的url都需要在url和options拼接后识别出来
'''
def generate_new_url(row):
    url=row['url']
    options_str=row['options']
    options_dict = json.loads(options_str)
    prodid = options_dict.get('prodid', '')
    RecoID = options_dict.get('RecoID', '')
    tabs = options_dict.get('tabs', '')
    type = options_dict.get('type', '')
    redirectId = options_dict.get('redirectId', '')
    query_parameters = []
    if tabs:    
        query_parameters.append(f"tabs={tabs}") 
    if prodid:
        query_parameters.append(f"prodid={prodid}")
    if RecoID:
        query_parameters.append(f"RecoID={RecoID}")
    if type:
        query_parameters.append(f"type={type}")
    if redirectId:
        query_parameters.append(f"redirectId={redirectId}")
    # 参数的添加顺序要变成先tabs，然后是redirectId然后是prodid然后才是RecoID

    if query_parameters:
        new_url = f"{url}?{'&'.join(query_parameters)}"
        return new_url
    else:
        return url
# def generate_new_url(row):
#     url = row['url']
#     options_str = row['options']
#     options_dict = json.loads(options_str)
#     prodid = options_dict.get('prodid', '')
#     RecoID = options_dict.get('RecoID', '')
#     tabs = options_dict.get('tabs', '')
#     query_parameters = []
#     if tabs:
#         query_parameters.append(f"tab={tabs}")
#     if prodid:
#         query_parameters.append(f"prodid={prodid}")
#     if RecoID:
#         query_parameters.append(f"RecoID={RecoID}")

#     if query_parameters:
#         new_url = f"{url}?{'&'.join(query_parameters)}"
#         print(new_url)
#         return new_url
#     else:
#         return url
    
def fake_generate_new_url(row):
    url = row['url']
    options_str = row['options']
    options_dict = json.loads(options_str)
    prodid = options_dict.get('prodid', '')
    RecoID = options_dict.get('RecoID', '')
    tabs = options_dict.get('tabs', '')
    query_parameters = []
    if tabs:
        query_parameters.append(f"tabs={tabs}")
    if prodid:
        query_parameters.append(f"prodid={prodid}")
    if RecoID:
        query_parameters.append(f"RecoID={RecoID}")

    if query_parameters:
        new_url = f"{url}?{'&'.join(query_parameters)}"
        return new_url
    else:
        return url
def get_data(df, click_data):
    df['手机号']= df['手机号'].astype(str)
    click_data1 = click_data[click_data["户外广告埋点"]==df['户外广告埋点'].unique()[0]]
    users1 = click_data1["手机号"].tolist()
    df["点击"] = df["手机号"].apply(lambda x: 1 if x in users1 else 0)
    return df
# 文件路径获取
def find_all_files(path):
    files = []
    for root, dirs, file in os.walk(path):
        for f in file:
            files.append(os.path.join(root, f))
    return files
# 获取用户名单
def users_list(gift_data,money_data,account_data):
    gift_id = gift_data["手机号"].unique().tolist()
    account_id = account_data["手机号"].unique().tolist()
    money_id = money_data["手机号"].unique().tolist()
    users_list = {
        'gift':gift_id,
        'account':account_id,
        'money':money_id,
    }
    return users_list
# 用户行为判断
def data_fliter(data,users):
    if data in users:
        return 1
    else:
        return 0
# 数据处理
def data_processing(mid_data,users_list):
        mid_data["兑礼"] = np.where(mid_data["手机号"].isin(users_list['gift']),1,0)
        mid_data["转化"] = np.where(mid_data["手机号"].isin(users_list['money']),1,0)
        mid_data["激活"] = np.where(mid_data["手机号"].isin(users_list['account']),1,0)
        mid_data["转化金额"].fillna(0,inplace=True)
        final_data =mid_data[['场景','人群名称','手机号','触达','点击', '兑礼','转化','转化金额','激活']]
        final_data.to_excel("final01.xlsx")
        # 统计不同人群名称的相关数据，并将结果存储到 final 中
        grouped1 = final_data.groupby(['场景','人群名称'])
        final1 = pd.DataFrame({
            '计划触达数': grouped1['手机号'].nunique(),
            '实际触达数': grouped1['触达'].sum(),
            '点击人数': grouped1['点击'].sum(),
            '点击率': (grouped1['点击'].sum()/grouped1['触达'].sum()).apply(lambda x: '{:.2%}'.format(x)),
            '转化人数': grouped1['转化'].sum(),
        }).reset_index()
        # final1['点击率'] = final1['点击人数']/final1['实际触达数']

        grouped2 = final_data[final_data["点击"]==1].groupby(['场景','人群名称'])
        final2 = pd.DataFrame({
            '点击后兑礼人数': grouped2['兑礼'].sum(),
            '点击后转化人数': grouped2['转化'].sum(),
            '点击后转化金额': grouped2["转化金额"].sum(),
            '点击后激活': grouped2["激活"].sum()
        }).reset_index()
        final2['点击后兑礼率'] = (final2["点击后兑礼人数"]/final1["点击人数"]).apply(lambda x: '{:.2%}'.format(x) if final1["点击人数"].any() != 0 else 0)
        final2['点击后转化率'] = (final2["点击后转化人数"]/final1["点击人数"]).apply(lambda x: '{:.2%}'.format(x) if final1["点击人数"].any() != 0 else 0)

        grouped3 = final_data[final_data["触达"]==1].groupby(['场景','人群名称'])
        final3 = pd.DataFrame({
            '整体兑礼人数': grouped3['兑礼'].sum(),
            '整体转化人数': grouped3['转化'].sum(),
            '整体转化金额': grouped3["转化金额"].sum(),
            '整体激活人数': grouped3['激活'].sum(),
        }).reset_index()
        final3['整体兑礼率'] = (final3["整体兑礼人数"]/final1["实际触达数"]).apply(lambda x: '{:.2%}'.format(x))
        final3['整体转化率'] = (final3["整体转化人数"]/final1["实际触达数"]).apply(lambda x: '{:.2%}'.format(x))
        merged = pd.merge(final1, final2, on=['人群名称','场景'], how='left')
        final = pd.merge(merged, final3, on=['人群名称','场景'], how='left')
        final.fillna(0,inplace=True)
        final['点击后转化金额']=final['点击后转化金额'].astype(int)
        final['整体转化金额']=final['点击后转化金额'].astype(int)
        return final
# 日期参数生成
params = {"start_date":start_date,"end_date":end_date}
# 数据源获取
filename = find_all_files('D:\\code\\沉睡触达\\org')
params

{'start_date': datetime.date(2023, 4, 1),
 'end_date': datetime.date(2023, 4, 25)}

# 导入数据库数据

In [18]:
# 数据库数据
with pymssql.connect(server, user, password, database) as conn:
    with conn.cursor(as_dict=True) as cursor:
        # 兑礼数据
        sql1 = f'''
            SELECT
                mobile AS 手机号,
                statuname AS 状态,
                date_add AS 日期,
                product_title AS 礼品 
            FROM
                v_mall_order_detail 
            WHERE
                title = '礼品' 
                AND date_add BETWEEN '{params['start_date']}' 
                    AND '{params['end_date']}' + ' 23:59:59'
                AND statuname != '关闭' 
                AND statuname != '申请退货' 
                AND statuname != '已退款' 
            ORDER BY
                date_add     
        '''
        cursor.execute(sql1)
        rows1 = cursor.fetchall()
        gift_data = pd.DataFrame(rows1, columns=['手机号', '状态', '日期', '礼品'])
        gift_data = gift_data[gift_data['状态'].isin(['关闭', '申请退货', '已退款']) == False]
        
        # 消费转化
        sql2 = f'''
            SELECT DISTINCT
                会员号 AS 手机号,
                sum(金额) AS 转化金额
            FROM
                BI_Business_Consume 
            WHERE
                消费时间 BETWEEN '{params['start_date']}' 
                    AND '{params['end_date']}' + ' 23:59:59'
            GROUP BY
                会员号 
        '''
        cursor.execute(sql2)
        rows2 = cursor.fetchall()
        money_data = pd.DataFrame(rows2, columns=['手机号', '转化金额'])
        money_data['手机号'] = money_data['手机号'].str.strip().astype(str)
        
        # 激活
        sql3 = f'''
            SELECT DISTINCT
                电话 AS 手机号 
            FROM
                BI_Business_Member 
            WHERE
                首次绑定时间 BETWEEN '{params['start_date']}' 
                    AND '{params['end_date']}' + ' 23:59:59'
        '''
        cursor.execute(sql3)
        rows3 = cursor.fetchall()
        account_data = pd.DataFrame(rows3, columns=['手机号'])
        account_data = account_data.drop_duplicates()
        # 点击数据
        sql4 = f'''
                SELECT
                    mobile as 手机号,url,options 
                FROM
                    [dbo].[BI_Business_Visit_Record_Extension] 
                WHERE
                    url like '%pages%'
                    and url like '%index%' 
                    AND date_add BETWEEN '{params['start_date']}' 
                    AND '{params['end_date']}' + ' 23:59:59'
                    AND options LIKE '%RecoID%' 
                    or options LIKE '%prodid%'
        '''
        cursor.execute(sql4)
        rows4 = cursor.fetchall()
        click_data = pd.DataFrame(rows4)
        click_data['户外广告埋点'] = click_data.apply(fake_generate_new_url, axis=1)
        click_data = click_data[['手机号','户外广告埋点']]

# 文件批量处理

In [60]:
# 读取 Excel 文件中的所有 Sheet
xls_file = pd.read_excel(filename[0], sheet_name=None,usecols = ["场景", "人群名称", "手机号", "户外广告埋点", "触达"])
# 遍历 Sheet 字典，将各个 Dataframe 存储在一个列表中
final_list = []
for sheet_name, df in xls_file.items():
    i = get_data(df, click_data)
    i["手机号"]= i["手机号"].astype(str)
    mid_data = i.merge(money_data,how='left',on="手机号")
    user_list = users_list(gift_data,money_data,account_data)
    final_list.append(mid_data)
    final_list.append(data_processing(mid_data,user_list))
# 创建一个 ExcelWriter 对象，指定要输出的 Excel 文件名
with pd.ExcelWriter('final_data.xlsx', engine='openpyxl') as writer:
    # 遍历 DataFrame 列表，将每个 DataFrame 输出到不同的工作表
    for index, df in enumerate(final_list):
        df.to_excel(writer, sheet_name=f'Sheet{index + 1}', index=False)


# 使用珠玑的点击数据的临时处理方案

In [163]:
# 读取目标文件数据
df1 = pd.read_excel('031301.xlsx')
df1 = df1[["场景","人群名称","手机号","触达","点击"]]
df1["手机号"]= df1["手机号"].astype(str)
mid_data1 = df1.merge(money_data,how='left',on="手机号")
# mid_data2 = df2.merge(money_data,how='left',on="手机号")
# mid_data3 = df3.merge(money_data,how='left',on="手机号")
user_list = users_list(gift_data,money_data,account_data)
import numpy as np
def data_processing(mid_data,users_list):
        mid_data["兑礼"] = np.where(mid_data["手机号"].isin(users_list['gift']),1,0)
        mid_data["转化"] = np.where(mid_data["手机号"].isin(users_list['money']),1,0)
        mid_data["激活"] = np.where(mid_data["手机号"].isin(users_list['account']),1,0)
        mid_data["转化金额"].fillna(0,inplace=True)
        final_data =mid_data[['场景','人群名称','手机号','触达','点击', '兑礼','转化','转化金额','激活']]
        final_data.to_excel("final01.xlsx")
        # 统计不同人群名称的相关数据，并将结果存储到 final 中
        grouped1 = final_data.groupby(['场景','人群名称'])
        final1 = pd.DataFrame({
            '计划触达数': grouped1['手机号'].nunique(),
            '实际触达数': grouped1['触达'].sum(),
            '点击人数': grouped1['点击'].sum(),
            '点击率': (grouped1['点击'].sum()/grouped1['触达'].sum()).apply(lambda x: '{:.2%}'.format(x)),
            '转化人数': grouped1['转化'].sum(),
        }).reset_index()
        # final1['点击率'] = final1['点击人数']/final1['实际触达数']

        grouped2 = final_data[final_data["点击"]==1].groupby(['场景','人群名称'])
        final2 = pd.DataFrame({
            '点击后兑礼人数': grouped2['兑礼'].sum(),
            '点击后转化人数': grouped2['转化'].sum(),
            '点击后转化金额': grouped2["转化金额"].sum(),
            '点击后激活': grouped2["激活"].sum()
        }).reset_index()
        final2['点击后兑礼率'] = final2["点击后兑礼人数"]/final1["点击人数"]
        final2['点击后转化率'] = final2["点击后转化人数"]/final1["点击人数"]

        grouped3 = final_data[final_data["触达"]==1].groupby(['场景','人群名称'])
        final3 = pd.DataFrame({
            '整体兑礼人数': grouped3['兑礼'].sum(),
            '整体转化人数': grouped3['转化'].sum(),
            '整体转化金额': grouped3["转化金额"].sum(),
            '整体激活人数': grouped3['激活'].sum(),
        }).reset_index()
        final3['整体兑礼率'] = final3["整体兑礼人数"]/final1["实际触达数"]
        final3['整体转化率'] = final3["整体转化人数"]/final1["实际触达数"]
        merged = pd.merge(final1, final2, on=['人群名称','场景'], how='inner')
        final = pd.merge(merged, final3, on=['人群名称','场景'], how='inner')
        return final
final1 = data_processing(mid_data1,user_list)
final1.to_excel("sheet1.xlsx")

In [ ]:
with pymssql.connect(server, user, password, database) as conn:
    with conn.cursor(as_dict=True) as cursor:
        # 点击数据
        sql4 = f'''
                SELECT
                    mobile as 手机号,url,options 
                FROM
                    [dbo].[BI_Business_Visit_Record_Extension] 
                WHERE
                    url like '%pages%'
                    and url like '%index%' 
                    AND date_add BETWEEN '2023-01-01' 
                    AND '2023-3-31' + ' 23:59:59'
                    AND options LIKE '%RecoID%' 
                    or options LIKE '%prodid%'
                    or options LIKE '%type%'
                    or options LIKE '%redirectId%'
        '''
        cursor.execute(sql4)
        rows4 = cursor.fetchall()
        click_data = pd.DataFrame(rows4)
        click_data["手机号"]= click_data["手机号"].astype(str)
        click_data['户外广告埋点'] = click_data.apply(generate_new_url, axis=1)
        click_data = click_data[['手机号','户外广告埋点']]